<a href="https://colab.research.google.com/github/youssoubo1/zadanie/blob/main/Copie_de_Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # Все пакеты, которые нам нужно импортировать 
import numpy as np               # для линейной алгебры
import pandas as pd              # для табличного вывода
from scipy.stats import rankdata # для ранжирования кандидатов

In [ ]:
# Данные закодированы в векторы и матрицы

attributes = np.array(["гибкость", "Управляемость", "Временные хар-ки", "Восстанавливаемость", "Покрытие контекста"])
candidates = np.array(["оzon", "amazon", "aliexpress", "wilberrу"])
raw_data = np.array([
    [0.43, 0.35,  0.46,  0.37,  0.23],
    [0.37, 0.44,  0.35,  0.32,  0.57],
    [0.38, 0.41,  0.41,  0.43,  0.40],
    [0.39, 0.43,  0.35,  0.53,  0.34],
])

weights = np.array([0.3, 0.2, 0.2, 0.15, 0.15])

# Индексы атрибутов (начиная с нуля), которые считаются полезными.
# Неупомянутые индексы считаются стоимостными атрибутами.
benefit_attributes = set([0, 1, 2, 3])

# Отобразить необработанные данные, которые у нас есть
pd.DataFrame(data=raw_data, index=candidates, columns=attributes)

,гибкость,Управляемость,Временные хар-ки,Восстанавливаемость,Покрытие контекста
оzon,0.43,0.35,0.46,0.37,0.23
amazon,0.37,0.44,0.35,0.32,0.57
aliexpress,0.38,0.41,0.41,0.43,0.40
wilberrу,0.39,0.43,0.35,0.53,0.34


####Расчет взвешенных нормализованных оценок

In [ ]:
m = len(raw_data)
n = len(attributes)
raw_data *= weights
pd.DataFrame(data=raw_data, index=candidates, columns=attributes)

,гибкость,Управляемость,Временные хар-ки,Восстанавливаемость,Покрытие контекста
оzon,0.129,0.070,0.092,0.0555,0.0345
amazon,0.111,0.088,0.070,0.0480,0.0855
aliexpress,0.114,0.082,0.082,0.0645,0.0600
wilberrу,0.117,0.086,0.070,0.0795,0.0510


In [ ]:
a_pos = np.zeros(n)
a_neg = np.zeros(n)
for j in range(n):
    column = raw_data[:,j]
    max_val = np.max(column)
    min_val = np.min(column)
    
    # # Посмотрите, хотим ли мы максимизировать выгоду или минимизировать затраты (для PIS)
    # Voir si nous voulons maximiser les avantages ou minimiser les coûts (pour PIS)
    if j in benefit_attributes:
        a_pos[j] = max_val
        a_neg[j] = min_val
    else:
        a_pos[j] = min_val
        a_neg[j] = max_val

pd.DataFrame(data=[a_pos, a_neg], index=["A*", "A-"], columns=attributes)

,гибкость,Управляемость,Временные хар-ки,Восстанавливаемость,Покрытие контекста
A*,0.129,0.088,0.092,0.0795,0.0345
A-,0.111,0.070,0.070,0.0480,0.0855


In [ ]:
sp = np.zeros(m)
sn = np.zeros(m)
cs = np.zeros(m)

for i in range(m):
    diff_pos = raw_data[i] - a_pos
    diff_neg = raw_data[i] - a_neg
    sp[i] = np.sqrt(diff_pos @ diff_pos)
    sn[i] = np.sqrt(diff_neg @ diff_neg)
    cs[i] = sn[i] / (sp[i] + sn[i])

pd.DataFrame(data=zip(sp, sn, cs), index=candidates, columns=["S*", "S-", "C*"])

,S*,S-,C*
оzon,0.030000,0.058866,0.662414
amazon,0.066342,0.018000,0.213417
aliexpress,0.035160,0.034921,0.498295
wilberrу,0.030071,0.049744,0.623245


In [ ]:
def rank_according_to(data):
    ranks = rankdata(data).astype(int)
    ranks -= 1
    return candidates[ranks][::-1]

In [ ]:
cs_order = rank_according_to(cs)
sp_order = rank_according_to(sp)
sn_order = rank_according_to(sn)

pd.DataFrame(data=zip(cs_order, sp_order, sn_order), index=range(1, m + 1), columns=["C*", "S*", "S-"])

,C*,S*,S-
1,aliexpress,amazon,aliexpress
2,amazon,aliexpress,amazon
3,оzon,wilberrу,оzon
4,wilberrу,оzon,wilberrу


In [ ]:
print("Лучший кандидат/альтернатива согласно C*:" + cs_order[0])
print("Предпочтения в порядке убывания " + " , ".join(cs_order) + ".")

Лучший кандидат/альтернатива согласно C*:aliexpress
Предпочтения в порядке убывания aliexpress , amazon , оzon , wilberrу.
